In [ ]:
# **Neuro RLs** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()
### **TReND Computational Neuroscience and Machine Learning Summer School, 2023**
#### made by: **Tom George, UCL**

In this tutorial we'll build and train Hopfield networks. By the end we'll have a network which can memorise flags from all 54 African Nations.

<center><img src="./images/hfn.gif" width=1000></center>